<a href="https://colab.research.google.com/github/Andru-1987/86400_data_science_i_diplomatura/blob/main/08_aprendizaje_no_supervisado/clase_teorica/encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoding de Variables Categóricas

## Tabla Comparativa de Métodos de Encoding

| **Método** | **Cuándo usar** | **Ventajas** | **Desventajas** | **Algoritmos compatibles** |
|------------|-----------------|--------------|-----------------|---------------------------|
| **Label Encoding** | Variables ordinales con orden natural (ej: bajo/medio/alto) | Simple, eficiente en memoria | Introduce orden artificial si no existe | Árboles de decisión, Random Forest, XGBoost |
| **One-Hot Encoding** | Pocas categorías (<10-15), sin orden natural | No introduce orden artificial, funciona bien con regresión | Alta dimensionalidad, muchas columnas dispersas | Regresión Lineal/Logística, SVM, Redes Neuronales |
| **Ordinal Encoding** | Variables con orden claro y definido | Preserva el orden semántico | Requiere conocimiento del dominio | Cualquier algoritmo |
| **Binary Encoding** | Muchas categorías (10-100), menos columnas que One-Hot | Balance entre dimensionalidad y información | Más complejo de interpretar | Árboles, Gradient Boosting |
| **Target/Mean Encoding** | Alta cardinalidad (100+ categorías), datos grandes | Muy eficiente, captura relación con target | Alto riesgo de overfitting, requiere validación cruzada | Gradient Boosting, XGBoost, LightGBM |
| **Frequency Encoding** | Alta cardinalidad, categorías con frecuencias significativas | Simple, reduce dimensionalidad | Pierde información si frecuencias son similares | Cualquier algoritmo |
| **Hash Encoding** | Altísima cardinalidad (1000+ categorías), text data | Dimensionalidad fija y controlada | Colisiones (categorías diferentes con mismo hash) | Todos, especialmente ML online |



**Árbol de decisión para elegir encoding:**

```
¿La variable tiene orden natural? (ej: bajo → medio → alto)
├─ SÍ → Ordinal Encoding
└─ NO → ¿Cuántas categorías únicas tiene?
    ├─ 2 categorías → Label Encoding
    ├─ 3-10 categorías → One-Hot Encoding
    ├─ 10-100 categorías → Binary o Target Encoding
    └─ 100+ categorías → Target o Hash Encoding
```

**Consideraciones finales:**

1. **Evita data leakage**: Siempre ajusta (fit) los encoders solo en el conjunto de entrenamiento
2. **Target Encoding con CV**: Usa validación cruzada para evitar overfitting
3. **Guarda tus encoders**: En producción, necesitas los mismos encoders para datos nuevos
4. **Prueba múltiples estrategias**: La mejor opción depende de tus datos y modelo

In [36]:
!pip install category-encoders

In [37]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
import category_encoders as ce  # https://pypi.org/project/category-encoders/
import warnings
warnings.filterwarnings('ignore')


In [38]:
data = {
    'educacion': ['Secundaria', 'Universitaria', 'Posgrado', 'Secundaria', 'Universitaria',
                  'Posgrado', 'Primaria', 'Universitaria', 'Posgrado', 'Secundaria'] * 10,
    'departamento': ['Ventas', 'IT', 'Marketing', 'IT', 'Ventas', 'HR', 'IT', 'Marketing',
                     'Ventas', 'HR'] * 10,
    'ciudad': ['Buenos Aires', 'Córdoba', 'Rosario', 'Santa Fe', 'Mendoza',
               'Buenos Aires', 'Córdoba', 'La Pampa', 'Buenos Aires', 'Córdoba'] * 10,
    'salario': np.random.randint(30000, 120000, 100)
}

df = pd.DataFrame(data)

In [39]:
print("DATASET ORIGINAL")

print(df.head(10))
print(f"\nShape: {df.shape}")
print(f"\nValores únicos por columna:")



DATASET ORIGINAL
       educacion departamento        ciudad  salario
0     Secundaria       Ventas  Buenos Aires    33272
1  Universitaria           IT       Córdoba   106449
2       Posgrado    Marketing       Rosario   106156
3     Secundaria           IT      Santa Fe    86849
4  Universitaria       Ventas       Mendoza    66734
5       Posgrado           HR  Buenos Aires   105838
6       Primaria           IT       Córdoba    74252
7  Universitaria    Marketing      La Pampa    91915
8       Posgrado       Ventas  Buenos Aires    32532
9     Secundaria           HR       Córdoba    89716

Shape: (100, 4)

Valores únicos por columna:


In [40]:
print("Muestra de categorias por columna")
for col in ['educacion', 'departamento', 'ciudad']:
    print(f"  {col}: {df[col].nunique()} categorías -> {df[col].unique()}")


Muestra de categorias por columna
  educacion: 4 categorías -> ['Secundaria' 'Universitaria' 'Posgrado' 'Primaria']
  departamento: 4 categorías -> ['Ventas' 'IT' 'Marketing' 'HR']
  ciudad: 6 categorías -> ['Buenos Aires' 'Córdoba' 'Rosario' 'Santa Fe' 'Mendoza' 'La Pampa']


### 1. LABEL ENCODING - Para variables ordinales o árboles de decisión

In [41]:
print("✓ Mejor para: Variables ordinales o modelos basados en árboles")
print("✗ Evitar en: Regresión lineal/logística (introduce orden artificial)")

df_label = df.copy()

le = LabelEncoder()

# Aplicar a todas las columnas categóricas
for col in ['educacion', 'departamento', 'ciudad']:
    df_label[f'{col}_label'] = le.fit_transform(df_label[col])

print("\nResultado:")
print(df_label[['educacion', 'educacion_label', 'departamento', 'departamento_label']].head())
print(f"\nNuevas columnas: {df_label.shape[1] - df.shape[1]}")
print("⚠️  Nota: 'Córdoba'=0, 'Buenos Aires'=1... introduce orden que NO existe")

✓ Mejor para: Variables ordinales o modelos basados en árboles
✗ Evitar en: Regresión lineal/logística (introduce orden artificial)

Resultado:
       educacion  educacion_label departamento  departamento_label
0     Secundaria                2       Ventas                   3
1  Universitaria                3           IT                   1
2       Posgrado                0    Marketing                   2
3     Secundaria                2           IT                   1
4  Universitaria                3       Ventas                   3

Nuevas columnas: 3
⚠️  Nota: 'Córdoba'=0, 'Buenos Aires'=1... introduce orden que NO existe


### 2. ORDINAL ENCODING - Para variables con orden natural



In [42]:
print("✓ Mejor para: Variables con orden claro (educación, calificaciones)")

## recomendar! :D
df_ordinal = df.copy()

# Definir el orden correcto
education_order = [['Primaria', 'Secundaria', 'Universitaria', 'Posgrado']]

oe = OrdinalEncoder(categories=education_order)

df_ordinal['educacion_ordinal'] = oe.fit_transform(df_ordinal[['educacion']])

print("\nResultado (con orden semántico correcto):")
print(df_ordinal[['educacion', 'educacion_ordinal']].drop_duplicates().sort_values('educacion_ordinal'))
print("\n✓ Ahora el orden tiene sentido: Primaria(0) < Secundaria(1) < Universitaria(2) < Posgrado(3)")


✓ Mejor para: Variables con orden claro (educación, calificaciones)

Resultado (con orden semántico correcto):
       educacion  educacion_ordinal
6       Primaria                0.0
0     Secundaria                1.0
1  Universitaria                2.0
2       Posgrado                3.0

✓ Ahora el orden tiene sentido: Primaria(0) < Secundaria(1) < Universitaria(2) < Posgrado(3)



### 3. ONE-HOT ENCODING - Para variables nominales con pocas categorías

In [43]:
print("✓ Mejor para: Variables nominales con pocas categorías (<15)")
print("✓ Ideal para: Regresión lineal/logística, SVM, Redes Neuronales")

df_onehot = df.copy()

# Método 1: Con pandas (más simple)
# df_onehot = pd.get_dummies(df_onehot, columns=['departamento'], prefix='dept', drop_first=True)
df_onehot = pd.get_dummies(df_onehot, columns=['departamento'], prefix='dept')
df_onehot["departamento"] = df["departamento"].astype('category')



print("\nResultado:")
print(df_onehot.head())
print(f"\nShape después de One-Hot: {df_onehot.shape}")
print(f"Columnas nuevas: {df_onehot.shape[1] - df.shape[1]}")
print("\n✓ No introduce orden artificial")
print("'drop_first=True' evita multicolinealidad (dummy variable trap)")


✓ Mejor para: Variables nominales con pocas categorías (<15)
✓ Ideal para: Regresión lineal/logística, SVM, Redes Neuronales

Resultado:
       educacion        ciudad  salario  dept_HR  dept_IT  dept_Marketing  \
0     Secundaria  Buenos Aires    33272    False    False           False   
1  Universitaria       Córdoba   106449    False     True           False   
2       Posgrado       Rosario   106156    False    False            True   
3     Secundaria      Santa Fe    86849    False     True           False   
4  Universitaria       Mendoza    66734    False    False           False   

   dept_Ventas departamento  
0         True       Ventas  
1        False           IT  
2        False    Marketing  
3        False           IT  
4         True       Ventas  

Shape después de One-Hot: (100, 8)
Columnas nuevas: 4

✓ No introduce orden artificial
'drop_first=True' evita multicolinealidad (dummy variable trap)


### 4. BINARY ENCODING - Para cardinalidad media


In [44]:

print("✓ Mejor para: 10-100 categorías (menos columnas que One-Hot)")

df_binary = df.copy()
be = ce.BinaryEncoder(cols=['ciudad'])
df_binary = be.fit_transform(df_binary)

print("\nResultado:")
print(df_binary.head(10))
print(f"\nShape: {df_binary.shape}")
print("Explicación: Convierte cada categoría a binario")
print("  Buenos Aires (0) -> 000")
print("  Córdoba (1)      -> 001")
print("  Mendoza (2)      -> 010")
print("  Rosario (3)      -> 011")
print(f"\n✓ Solo necesita log2(n) columnas: {np.ceil(np.log2(df['ciudad'].nunique()))} en vez de {df['ciudad'].nunique()}")


✓ Mejor para: 10-100 categorías (menos columnas que One-Hot)

Resultado:
       educacion departamento  ciudad_0  ciudad_1  ciudad_2  salario
0     Secundaria       Ventas         0         0         1    33272
1  Universitaria           IT         0         1         0   106449
2       Posgrado    Marketing         0         1         1   106156
3     Secundaria           IT         1         0         0    86849
4  Universitaria       Ventas         1         0         1    66734
5       Posgrado           HR         0         0         1   105838
6       Primaria           IT         0         1         0    74252
7  Universitaria    Marketing         1         1         0    91915
8       Posgrado       Ventas         0         0         1    32532
9     Secundaria           HR         0         1         0    89716

Shape: (100, 6)
Explicación: Convierte cada categoría a binario
  Buenos Aires (0) -> 000
  Córdoba (1)      -> 001
  Mendoza (2)      -> 010
  Rosario (3)      -> 011

### 5. TARGET ENCODING - Para alta cardinalidad

In [45]:
print("✓ Mejor para: 100+ categorías, problemas de clasificación/regresión")
print(" IMPORTANTE: Aplicar con validación cruzada para evitar overfitting")

# Crear target variable (salario alto/bajo)
df_target = df.copy()
df_target['salario_alto'] = (df_target['salario'] > df_target['salario'].median()).astype(int)

# Split para evitar data leakage
X_train, X_test, y_train, y_test = train_test_split(
    df_target[['ciudad']],
    df_target['salario_alto'],
    test_size=0.3,
    random_state=42
)

# Target encoding con regularización
te = ce.TargetEncoder(cols=['ciudad'], smoothing=1.0)
X_train_te = te.fit_transform(X_train, y_train)
X_test_te = te.transform(X_test)

print("\nResultado (Train set):")
print(X_train_te.head(10))
print("\nPromedio de 'salario_alto' por ciudad:")
df_target.groupby('ciudad')['salario_alto'].mean().sort_values(ascending=False)
print("\n✓ Cada ciudad se reemplaza por la proporción promedio del target")
print("NUNCA usar el test set para calcular estos promedios (data leakage)")


✓ Mejor para: 100+ categorías, problemas de clasificación/regresión
 IMPORTANTE: Aplicar con validación cruzada para evitar overfitting

Resultado (Train set):
      ciudad
11  0.521386
47  0.514285
85  0.505960
28  0.505960
93  0.514286
5   0.505960
66  0.521386
65  0.505960
35  0.505960
16  0.521386

Promedio de 'salario_alto' por ciudad:

✓ Cada ciudad se reemplaza por la proporción promedio del target
NUNCA usar el test set para calcular estos promedios (data leakage)


### 6. FREQUENCY ENCODING - Simple y efectivo


In [46]:

print("✓ Mejor para: Cuando la frecuencia de la categoría es informativa")

df_freq = df.copy()

# Calcular frecuencias
freq_map = df_freq['ciudad'].value_counts(normalize=True).to_dict()
df_freq['ciudad_freq'] = df_freq['ciudad'].map(freq_map)

print("\nResultado:")
print(df_freq[['ciudad', 'ciudad_freq']].head(10))
print("\nFrecuencias:")
print(df_freq['ciudad'].value_counts(normalize=True))
print("\n✓ Simple y efectivo cuando la frecuencia es predictiva")


✓ Mejor para: Cuando la frecuencia de la categoría es informativa

Resultado:
         ciudad  ciudad_freq
0  Buenos Aires          0.3
1       Córdoba          0.3
2       Rosario          0.1
3      Santa Fe          0.1
4       Mendoza          0.1
5  Buenos Aires          0.3
6       Córdoba          0.3
7      La Pampa          0.1
8  Buenos Aires          0.3
9       Córdoba          0.3

Frecuencias:
ciudad
Buenos Aires    0.3
Córdoba         0.3
Rosario         0.1
Santa Fe        0.1
Mendoza         0.1
La Pampa        0.1
Name: proportion, dtype: float64

✓ Simple y efectivo cuando la frecuencia es predictiva


### 7. HASH ENCODING - Para cardinalidad extrema

In [47]:
print("✓ Mejor para: 1000+ categorías, datos de texto, ML online")

df_hash = df.copy()
he = ce.HashingEncoder(cols=['departamento', 'ciudad'], n_components=4)
df_hash = he.fit_transform(df_hash)

print("\nResultado:")
print(df_hash.head())
print(f"\nShape: {df_hash.shape}")
print("✓ Dimensionalidad fija (n_components=4) sin importar cuántas categorías")
print("Puede haber colisiones (2 categorías con mismo hash)")

✓ Mejor para: 1000+ categorías, datos de texto, ML online

Resultado:
   col_0  col_1  col_2  col_3      educacion  salario
0      2      0      0      0     Secundaria    33272
1      1      0      0      1  Universitaria   106449
2      1      0      0      1       Posgrado   106156
3      1      0      1      0     Secundaria    86849
4      1      0      0      1  Universitaria    66734

Shape: (100, 6)
✓ Dimensionalidad fija (n_components=4) sin importar cuántas categorías
Puede haber colisiones (2 categorías con mismo hash)
